## IMPORTS 

In [338]:
import random
import pandas as pd
import numpy as np
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torchtext.data import Example
from sklearn.metrics import f1_score
import torchtext


SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# load spacy tokenizer
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])

## LOAD PROCESSED TRAINING DATA FROM DISK

In [339]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

In [340]:
sin = len(df_train[df_train["target"]==0])
insin = len(df_train[df_train["target"]==1])
persin = (sin/(sin+insin))*100
perinsin = (insin/(sin+insin))*100            
print("# insincere questions: {:,}({:.2f}%) and # sincere questions: {:,}({:.2f}%)".format(sin,persin,insin,perinsin))
# print("Sinsere:{}% Insincere: {}%".format(round(persin,2),round(perinsin,2)))
print("# Test samples: {:,}({:.3f}% of train samples)".format(len(df_test),len(df_test)/len(df_train)))

# insincere questions: 1,225,312(93.81%) and # sincere questions: 80,810(6.19%)
# Test samples: 56,370(0.043% of train samples)


## Normalization

In [341]:
df_train["question_text"] = df_train["question_text"].fillna("_na_").values
df_test["question_text"] = df_test["question_text"].fillna("_na_").values

df_train.to_csv("train2.csv")
df_test.to_csv("test2.csv")

In [342]:
df_train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


## Create a pytorch dataset from the train samples and build a vocabulary using embedding vectors

In [343]:
# # load dataframe to csv

TEXT = data.Field(lower=True, batch_first=True,tokenize='spacy')#preprocessing=generate_bigrams)
LABEL = data.LabelField(dtype=torch.float)
qid = data.Field()

train_dataset = data.TabularDataset(path='train2.csv', format='csv',
                                      fields={'question_text': ('text',TEXT),
                                              'target': ('label',LABEL)})
final_test_dataset = data.TabularDataset(path='test2.csv', format='csv',
                                     fields={'qid': ('qid', qid),
                                             'question_text': ('text', TEXT)})

In [344]:
TEXT.build_vocab(train_dataset, final_test_dataset, min_freq=3)
qid.build_vocab(final_test_dataset)

In [345]:
import torchtext
vec = torchtext.vocab.Vectors('../input/embeddings/glove.840B.300d/glove.840B.300d.txt', cache='./cache/')
# vec = torchtext.vocab.Vectors('wiki-news-300d-1M/wiki-news-300d-1M.vec', cache='./cache/')
TEXT.vocab.load_vectors(vec)

In [346]:
# TEXT.build_vocab(train_dataset, max_size=50000, vectors=vec)
LABEL.build_vocab(train_dataset)
TEXT.vocab.vectors.shape

torch.Size([72017, 300])

## SPLIT DATA TO TRAINiNG AND VALIDATION SETS

In [357]:
# train_dataset, test_dataset = train_dataset.split(split_ratio=0.9,random_state=random.seed(SEED))
train_dataset, valid_dataset = train_dataset.split(split_ratio=0.9,random_state=random.seed(SEED))

In [358]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.BucketIterator(
    train_dataset, 
    batch_size=BATCH_SIZE,sort_key=lambda x: len(x.text),shuffle=True,sort=False, 
    device=device)
# test_iterator = data.BucketIterator(
#     test_dataset, 
#     batch_size=BATCH_SIZE,sort_key=lambda x: len(x.text),train=False,sort=False,
#     device=device)
valid_iterator = data.BucketIterator(
    valid_dataset, 
    batch_size=BATCH_SIZE,sort_key=lambda x: len(x.text),train=False,sort=False, 
    device=device)

In [374]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class Attention(nn.Module):
	'''
	Class to define the attention network component (attention is computed over the entire sequence length so
	that the information from each time step can be used in the final prediction and not just the final time step)
	The network is a 3 layer MLP architecture followed by a softmax
	Arguments:
		seq_len : maximum sequence length (this is required to predefine the dimensions of the network)
		hidden_emb : same as the hidden dimension of the lstm network
	Returns:
		None
	'''
	def __init__(self, seq_len=18, hidden_emb=1024):
		super(Attention, self).__init__()

		self.seq_len = seq_len
		self.hidden_emb = hidden_emb
		self.mlp1_units = 3072
		self.mlp2_units = 1024

		self.fc = nn.Sequential(
            nn.Linear(self.seq_len*self.hidden_emb, self.mlp1_units),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(self.mlp1_units, self.mlp2_units),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(self.mlp2_units, self.seq_len),
            nn.ReLU(inplace=True),
            )

		self.softmax = nn.Softmax(dim=1)

	'''
	Computes the attention on the lstm outputs from each time step in the sequence
	Arguments:
		lstm_emd : lstm embedding from each time step in the sequence
	Returns:
		attn_feature_map : embedding computed after applying attention to the lstm embedding of the entire sequence
	'''
	def forward(self, lstm_emd):

		batch_size = lstm_emd.shape[0]
		lstm_emd = lstm_emd.contiguous()
		lstm_flattened = lstm_emd.view(batch_size, -1) # to pass it to the MLP architecture

		attn = self.fc(lstm_flattened) # attention over the sequence length
		alpha = self.softmax(attn) # gives the probability values for the time steps in the sequence (weights to each time step)

		alpha = torch.stack([alpha]*self.mlp2_units, dim=2) # stack it across the lstm embedding dimesion

		attn_feature_map = lstm_emd * alpha # gives attention weighted lstm embedding
		attn_feature_map = torch.sum(attn_feature_map, dim=1, keepdim=True) # computes the weighted sum
		return attn_feature_map

In [359]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=100):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [360]:
class BiLSTM(nn.Module):
    def __init__(self, pretrained_lm, padding_idx, static=True, hidden_dim=128, lstm_layer=2, dropout=0.2):
        super(BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(p=dropout)
        self.embedding = nn.Embedding.from_pretrained(pretrained_lm)
        self.embedding.padding_idx = padding_idx
        if static:
            self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=lstm_layer, 
                            dropout = dropout,
                            bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*lstm_layer*2, 1)
        self.fc = nn.Linear(128*2, 1)
        self.attention = SelfAttention(128*2, batch_first=True)

    def forward(self, sents):
        x = self.embedding(sents)
        x = self.dropout(x)
        x = torch.transpose(x, dim0=1, dim1=0)
#         self.out = nn.Linear(hidden_dim, num_classes)
        lstm_out, (h_n, c_n) = self.lstm(x)
        y = self.hidden2label(self.dropout(torch.cat([c_n[i,:, :] for i in range(c_n.shape[0])], dim=1)))
        return y    

In [361]:
import math
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
OUTPUT_DIM = 1
DROPOUT = 0.5

# model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)
model = BiLSTM(TEXT.vocab.vectors, lstm_layer=2, padding_idx=TEXT.vocab.stoi[TEXT.pad_token], hidden_dim=128,dropout=0.2).cuda()
print(model)

BiLSTM(
  (dropout): Dropout(p=0.2)
  (embedding): Embedding(72017, 300, padding_idx=1)
  (lstm): LSTM(300, 128, num_layers=2, dropout=0.2, bidirectional=True)
  (hidden2label): Linear(in_features=512, out_features=1, bias=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (attention): SelfAttention()
)


In [362]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0869,  0.1916,  0.1091,  ..., -0.0152,  0.1111,  0.2065],
        ...,
        [-0.2523, -0.1560, -0.0008,  ...,  0.0601,  0.3452,  0.2371],
        [ 0.0202,  0.1975, -0.0793,  ...,  0.0901, -0.6364,  0.2416],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

### Training

In [363]:
import torch.optim as optim

# optimizer = optim.Adam(model.parameters())
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y, th = 0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

def f1_score_model(preds, y,th = 0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded = torch.sigmoid(preds).cpu().apply_(lambda x: 1 if x>=th else 0)
    rounded_preds = rounded

    return f1_score(y.cpu().numpy(),rounded_preds.cpu().numpy())


In [364]:
stop_training = 0
warmup_epoch = 2
step = 0
max_loss = 1e5
no_improve_in_previous_epoch = False
no_improve_epoch = 0
fine_tuning = False
train_record = []
val_record = []
losses = []

def train(model, iterator, optimizer, criterion,e):
    
    epoch_loss = 0
    epoch_acc = 0

    global warm_epoch
    global no_improve_in_previous_epoch
    global fine_tuning
    global step
    global max_loss
    global stop_training
    global no_improve_epoch
    global train_record
    global val_record
    global losses
    
    model.train()
    if e >= warmup_epoch:
        if no_improve_in_previous_epoch:
            no_improve_epoch += 1
            if no_improve_epoch >= 1:
                stop_training = 1
        else:
            no_improve_epoch = 0
        no_improve_in_previous_epoch = True
    if stop_training == 0:    
        if not fine_tuning and e >= warmup_epoch:
            model.embedding.weight.requires_grad = True        
            fine_tuning = True

        for batch in iterator:
            step += 1
            model.train()
            optimizer.zero_grad()
            predictions = model(batch.text).squeeze(1)       

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            loss.backward()

            optimizer.step()
            if step % 500 == 0:
                model.eval()
                model.zero_grad()
                val_loss = []
                for val_batch in iter(valid_iterator):
                    val_x = val_batch.text.cuda()
                    val_y = val_batch.label.type(torch.Tensor).cuda()
                    val_pred = model.forward(val_x).view(-1)
                    val_loss.append(criterion(val_pred, val_y).cpu().data.numpy())
                val_record.append({'step': step, 'loss': np.mean(val_loss)})
                print('epoch {:02} - step {:06} - train_loss {:.4f} - val_loss {:.4f} '.format(
                            e+1, step, np.mean(losses), val_record[-1]['loss']))
                if e >= warmup_epoch:
                    if val_record[-1]['loss'] <= max_loss:
                        save(m=model, info={'step': step, 'epoch': e+1, 'train_loss': np.mean(losses),
                                            'val_loss': val_record[-1]['loss']})
                        max_loss = val_record[-1]['loss']
                        no_improve_in_previous_epoch = False

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_f1(model, iterator, criterion,th=0.5):
    
    f1_scores = []
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            f1 = f1_score_model(predictions, batch.label,th=th)
            f1_scores.append(f1)
        
    return np.array(f1_scores).mean()


def save(m, info):
    torch.save(info, 'best_model.info')
    torch.save(m, 'best_model.m')
    
def load():
    m = torch.load('best_model.m')
    info = torch.load('best_model.info')
    return m, info

In [365]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion,e=epoch)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')
    if stop_training == 1:
        break

epoch 01 - step 000500 - train_loss nan - val_loss 0.1190 
epoch 01 - step 001000 - train_loss nan - val_loss 0.1153 
epoch 01 - step 001500 - train_loss nan - val_loss 0.1090 
epoch 01 - step 002000 - train_loss nan - val_loss 0.1067 
| Epoch: 01 | Train Loss: 0.120 | Train Acc: 95.34% | Val. Loss: 0.107 | Val. Acc: 95.78% |
epoch 02 - step 002500 - train_loss nan - val_loss 0.1050 
epoch 02 - step 003000 - train_loss nan - val_loss 0.1052 
epoch 02 - step 003500 - train_loss nan - val_loss 0.1035 
epoch 02 - step 004000 - train_loss nan - val_loss 0.1027 
| Epoch: 02 | Train Loss: 0.104 | Train Acc: 95.87% | Val. Loss: 0.102 | Val. Acc: 95.94% |
epoch 03 - step 004500 - train_loss nan - val_loss 0.1023 
epoch 03 - step 005000 - train_loss nan - val_loss 0.1007 
epoch 03 - step 005500 - train_loss nan - val_loss 0.1013 
epoch 03 - step 006000 - train_loss nan - val_loss 0.1003 
| Epoch: 03 | Train Loss: 0.099 | Train Acc: 96.06% | Val. Loss: 0.100 | Val. Acc: 96.02% |
epoch 04 - step 

In [366]:
model, m_info = load()
m_info

{'step': 10000, 'epoch': 5, 'train_loss': nan, 'val_loss': 0.097822756}

In [367]:
test_loss, test_acc = evaluate(model, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.098 | Test Acc: 96.13% |


In [368]:
score = evaluate_f1(model, valid_iterator, criterion)
print(f'| f1 score: {score:.3f}')

| f1 score: 0.645


In [369]:
model.eval()
val_pred = []
val_true = []
valid_iterator.init_epoch()
for val_batch in iter(valid_iterator):
    val_x = val_batch.text.cuda()
    val_true += val_batch.label.cpu().data.numpy().tolist()
    val_pred += torch.sigmoid(model.forward(val_x).view(-1)).cpu().data.numpy().tolist()

In [370]:
tmp = [0,0,0] # idx, cur, max
delta = 0
for tmp[0] in np.arange(0.1, 0.501, 0.01):
    tmp[1] = f1_score(val_true, np.array(val_pred)>tmp[0])
    if tmp[1] > tmp[2]:
        delta = tmp[0]
        tmp[2] = tmp[1]
print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))

best threshold is 0.3300 with F1 score: 0.6799


In [371]:
model.eval()
model.zero_grad()
test_pred = []
test_id = []

final_test_iterator = torchtext.data.BucketIterator(dataset=final_test_dataset,
                                    batch_size=BATCH_SIZE,
                                    sort_key=lambda x: x.text.__len__(),train=False,sort=False)


for test_batch in iter(final_test_iterator):
    test_x = test_batch.text.cuda()
    test_pred += torch.sigmoid(model.forward(test_x).view(-1)).cpu().data.numpy().tolist()
    test_id += test_batch.qid.view(-1).data.numpy().tolist()

In [372]:
sub_df =pd.DataFrame()
sub_df['qid'] = [qid.vocab.itos[i] for i in test_id]
sub_df['prediction'] = (np.array(test_pred) >= delta).astype(int)

In [373]:
sub_df.to_csv("submission.csv", index=False)